# Thermostat demo

In [1]:
!pip3 install --upgrade pip
!pip3 install cmake
!pip3 install cython
!pip3 install numpy
!pip3 install torch
!pip3 install datasets
!pip3 install spacy
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install overrides
!pip3 install jsonnet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
# Include root directory in module path
sys.path.append('src')

from pprint import pprint

import thermostat  # Accompanying library

# Load dataset

This will use the dataset script ("thermostat.py") in the "thermostat" directory.
In this example, we use the test set of the "imdb-bert-lig" configuration.
This refers to Layer Integrated Gradients (LIG) explanations of the predictions by a BERT model that has been fine-tuned on the IMDb (train) dataset and evaluated on the IMDb test dataset.
In other words, we load the 25k test examples from the IMDb test plus the BERT predictions and the feature attributions from a Layer Integrated Gradients explainer.

In [4]:
data = thermostat.load("imdb-bert-lig")
print(data)

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/imdb-bert-lig/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: imdb-bert-lig
IMDb dataset, BERT model, Layer Integrated Gradients explanations
Explainer: LayerIntegratedGradients
Model: textattack/bert-base-uncased-imdb
Dataset: imdb



Now let's inspect a single instance of the loaded dataset.
Here, we will stick to the functionality that the datasets library already supplies us with.
For readability purposes, we will not print the whole content of that instance.
Instead, we're showing only the first few entries of the attributions and the input_ids, respectively.

In [5]:
example = data[0]

print(f'Index: {example.index}')
print(f'Attributions (first 4): {example.attributions[:4]}')
print(f'True label: {example.true_label}')
print(f'Predicted label: {example.predicted_label}')

Tokenizing imdb-bert-lig instances (Tokenizer: textattack/bert-base-uncased-imdb): 100%|██████████| 25000/25000 [00:13<00:00, 1817.36it/s]

Index: 0
Attributions (first 4): [0.0, -0.18146862089633942, 0.13141055405139923, 0.2554049491882324]
True label: {'index': 1, 'name': 'pos'}
Predicted label: {'index': 1, 'name': 'pos'}


In [6]:
print(f'Accuracy of {data.model_name} :')
sum([b_i.true_label == b_i.predicted_label for b_i in data])/len(data)

Accuracy of textattack/bert-base-uncased-imdb :


0.93028

In [7]:
print(f'Heatmap:\n{example.heatmap}')

Heatmap:
i (Index: 1, Score: -0.117, Color: #e1e1ff, Text field: text)
went (Index: 2, Score: 0.085, Color: #ffe9e9, Text field: text)
and (Index: 3, Score: 0.165, Color: #ffd4d4, Text field: text)
saw (Index: 4, Score: 0.036, Color: #fff5f5, Text field: text)
this (Index: 5, Score: -0.03, Color: #f7f7ff, Text field: text)
movie (Index: 6, Score: 0.533, Color: #ff7777, Text field: text)
last (Index: 7, Score: 0.24, Color: #ffc1c1, Text field: text)
night (Index: 8, Score: 0.171, Color: #ffd3d3, Text field: text)
after (Index: 9, Score: -0.045, Color: #f3f3ff, Text field: text)
being (Index: 10, Score: -0.01, Color: #fcfcff, Text field: text)
coaxed (Index: [11, 12], Score: 0.017, Color: #fffafa, Text field: text)
to (Index: 13, Score: 0.027, Color: #fff8f8, Text field: text)
by (Index: 14, Score: -0.021, Color: #f9f9ff, Text field: text)
a (Index: 15, Score: 0.008, Color: #fffdfd, Text field: text)
few (Index: 16, Score: 0.022, Color: #fff9f9, Text field: text)
friends (Index: 17, Scor

# Visualize data
Can we make this a bit more readable?  
Of course!  


In [9]:
example.render()

# Inspecting misclassified instances

In [10]:
misclassified = [instance for instance in data if instance.true_label != instance.predicted_label]
mc0 = misclassified[0]
print(f'True Label: {mc0.true_label}, Predicted Label: {mc0.predicted_label}')
mc0.render()

True Label: {'index': 1, 'name': 'pos'}, Predicted Label: {'index': 0, 'name': 'neg'}


In [18]:
against_pred = sorted([token for token in mc0.heatmap if token.attribution < -0.2], key=lambda t: t.attribution)
print(f'Highest scoring tokens for class {mc0.true_label}, "against the predicted label"')
thermostat.Heatmap(against_pred).render()

Highest scoring tokens for class {'index': 1, 'name': 'pos'}, "against the predicted label"


In [19]:
for_pred = sorted([token for token in mc0.heatmap if token.attribution > 0.2], key=lambda t: t.attribution,
                  reverse=True)
print(f'Highest scoring tokens for class {mc0.predicted_label}, "for the predicted label"')
thermostat.Heatmap(for_pred).render()

Highest scoring tokens for class {'index': 0, 'name': 'neg'}, "for the predicted label"


# Aggregate data
Let us first compare the heatmaps of two different models on the same dataset+explainer configuration, MNLI + Occlusion.

In [3]:
albert = thermostat.load("xnli-albert-occ")
bert = thermostat.load("xnli-bert-occ")
electra = thermostat.load("xnli-electra-occ")
roberta = thermostat.load("xnli-roberta-occ")
xlnet = thermostat.load("xnli-xlnet-occ")

Loading Thermostat configuration: xnli-albert-occ


Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/xnli-albert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e. Subsequent calls will reuse this data.
Loading Thermostat configuration: xnli-bert-occ


Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/xnli-bert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e. Subsequent calls will reuse this data.
Loading Thermostat configuration: xnli-electra-occ


Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/xnli-electra-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e. Subsequent calls will reuse this data.
Loading Thermostat configuration: xnli-roberta-occ


Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/xnli-roberta-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e. Subsequent calls will reuse this data.
Loading Thermostat configuration: xnli-xlnet-occ


Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/xnli-xlnet-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e. Subsequent calls will reuse this data.


In [4]:
for model_name, data in zip(["albert", "bert", "electra", "roberta", "xlnet"],
                            [albert, bert, electra, roberta, xlnet]):
    print(model_name, sum([m_i.true_label == m_i.predicted_label for m_i in data])/len(data))

Tokenizing xnli-albert-occ instances (Tokenizer: prajjwal1/albert-base-v2-mnli): 100%|██████████| 5010/5010 [00:05<00:00, 999.69it/s]
Tokenizing xnli-bert-occ instances (Tokenizer: textattack/bert-base-uncased-MNLI):   0%|          | 0/5010 [00:00<?, ?it/s]

albert 0.07125748502994012


Tokenizing xnli-bert-occ instances (Tokenizer: textattack/bert-base-uncased-MNLI): 100%|██████████| 5010/5010 [00:04<00:00, 1056.88it/s]
Tokenizing xnli-electra-occ instances (Tokenizer: howey/electra-base-mnli):   0%|          | 0/5010 [00:00<?, ?it/s]

bert 0.07045908183632735


Tokenizing xnli-electra-occ instances (Tokenizer: howey/electra-base-mnli): 100%|██████████| 5010/5010 [00:04<00:00, 1024.65it/s]
Tokenizing xnli-roberta-occ instances (Tokenizer: textattack/roberta-base-MNLI):   0%|          | 0/5010 [00:00<?, ?it/s]

electra 0.8830339321357286


Tokenizing xnli-roberta-occ instances (Tokenizer: textattack/roberta-base-MNLI): 100%|██████████| 5010/5010 [00:05<00:00, 944.71it/s]
Tokenizing xnli-xlnet-occ instances (Tokenizer: textattack/xlnet-base-cased-MNLI):   0%|          | 0/5010 [00:00<?, ?it/s]

roberta 0.30379241516966066


Tokenizing xnli-xlnet-occ instances (Tokenizer: textattack/xlnet-base-cased-MNLI): 100%|██████████| 5010/5010 [00:05<00:00, 990.13it/s]

xlnet 0.06367265469061877


In [17]:
model_miscls = {}
miscl_indices = []

for model_data in [albert, bert, electra, roberta, xlnet]:
    miscl_indices += [instance.index for instance in model_data if instance.true_label != instance.predicted_label]
    
miscl_indices = sorted(list(set(miscl_indices)))

for model_data in [albert, bert, electra, roberta, xlnet]:
    model_miscls[model_data.config_name] = [instance for instance in model_data if instance.index in miscl_indices]
    

[0, 2, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 59, 60, 61, 62, 65, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 127, 128, 129, 130, 131, 132, 134, 135, 136, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 151, 152, 153, 154, 155, 156, 157, 159, 161, 163, 164, 165, 167, 169, 170, 171, 173, 174, 177, 178, 179, 181, 182, 183, 184, 185, 187, 188, 189, 190, 192, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 212, 213, 214, 215, 217, 219, 220, 222, 224, 225, 227, 228, 229, 230, 231, 232, 233, 234, 236, 238, 239, 241, 242, 243, 245, 247, 249, 250, 251, 252, 254, 255, 256, 257, 258, 259, 261, 262, 263, 264, 265, 267, 269, 270, 274, 275, 276, 278, 279, 280, 281, 2

In [ ]:
import pandas as pd



In [4]:
albert[1].render()
albert[1].true_label, albert[1].predicted_label

Tokenizing multi_nli-albert-occ instances (Tokenizer: prajjwal1/albert-base-v2-mnli): 100%|██████████| 9815/9815 [00:07<00:00, 1345.79it/s]

INFO:root:Heatmap "premise"


INFO:root:Heatmap "hypothesis"


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

In [5]:
bert[1].render()
bert[1].true_label, bert[1].predicted_label

Tokenizing multi_nli-bert-occ instances (Tokenizer: textattack/bert-base-uncased-MNLI): 100%|██████████| 9815/9815 [00:06<00:00, 1415.68it/s]

INFO:root:Heatmap "premise"


INFO:root:Heatmap "hypothesis"


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

In [6]:
electra[1].render()
electra[1].true_label, electra[1].predicted_label

Tokenizing multi_nli-electra-occ instances (Tokenizer: howey/electra-base-mnli): 100%|██████████| 9815/9815 [00:06<00:00, 1450.00it/s]

INFO:root:Heatmap "premise"


INFO:root:Heatmap "hypothesis"


({'index': 2, 'name': 'contradiction'}, {'index': 2, 'name': 'contradiction'})

In [7]:
roberta[1].render()
roberta[1].true_label, roberta[1].predicted_label

Tokenizing multi_nli-roberta-occ instances (Tokenizer: textattack/roberta-base-MNLI): 100%|██████████| 9815/9815 [00:07<00:00, 1301.06it/s]

INFO:root:Heatmap "premise"


INFO:root:Heatmap "hypothesis"


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

In [8]:
xlnet[1].render()
xlnet[1].true_label, xlnet[1].predicted_label

Tokenizing multi_nli-xlnet-occ instances (Tokenizer: textattack/xlnet-base-cased-MNLI): 100%|██████████| 9815/9815 [00:07<00:00, 1353.55it/s]

INFO:root:Heatmap "premise"


INFO:root:Heatmap "hypothesis"


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

### Rank correlation
We can also investigate how well two explainers on the same dataset+model combination correlate regarding their attribution scores.